In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.special import gammaln
from scipy import stats
from scipy import integrate

figdir = Path('../assets/img/2019-08-22-classifying-columns/').resolve()

In [ ]:

plt.rcParams['font.size'] = 15
for xy in ['x', 'y']:
    plt.rcParams[f'{xy}tick.labelsize'] = 15
    plt.rcParams[f'{xy}tick.major.size'] = 5
    plt.rcParams[f'{xy}tick.major.width'] = 1

In [ ]:
def beta_log_normalizer(a, b):
    return gammaln(a) + gammaln(b) - gammaln(a + b)


def beta_binomial_lpdf(n, k, a, b):
    return beta_log_normalizer(a + k, b + n - k) - beta_log_normalizer(a, b)


def bernoulli_lpdf(n, k, p):
    return k * np.log(p) + (n - k) * np.log(1. - p)

# Beta Distribution

In [ ]:
pi = 0.3
nuvals = np.logspace(1, 4, 4)


x = np.linspace(0, 1, 200)

fig, axes = plt.subplots(ncols=len(nuvals), figsize=(len(nuvals) * 4, 3), sharey=True, gridspec_kw=dict(wspace=.05))

show_labels = True
for ax, nu in zip(axes, nuvals):
    alpha = pi * nu
    beta = (1 - pi) * nu
    ax.fill_between(x, stats.beta(alpha, beta).pdf(x))
    ax.text(.99, .02, '$\\nu = %d$' % nu, ha='right', va='bottom', transform=ax.transAxes)
    
    for k, spine in ax.spines.items():
        spine.set_visible(k == 'bottom')
    ax.set_yticks([])
    ax.set_xbound([0, 1])
    ax.set_ybound([0, None])
    ax.set_xticks([0, pi, 1])    
    if show_labels:
        ax.set_xticklabels(['0', '$\pi=%.1f$' % p, '1'])
        show_labels = False
    else:
        ax.set_xticklabels([])
        
fig.tight_layout()
fig.savefig(figdir / 'beta-distributions.jpg', dpi=300)

# Different regimes as number of observations scales

In [ ]:

p = .3
n = np.logspace(0, 4.1, 200)
k = p * n

evidence = beta_log_normalizer(k + 1, n - k + 1)
# evidence = bernoulli_lpdf(n, k, p)
fig, axes = plt.subplots(ncols=2, figsize=(15, 4), gridspec_kw=dict(wspace=0))
nu = 10.
pivals = [.3, .4, .5, .6]
ax = axes[0]
ax.set_xlabel('number of observations $n$')
ax.set_ylabel('likelihood')
for pi in pivals:
    a, b = pi * nu, (1 - pi) * nu
    y = beta_binomial_lpdf(n, k, a, b)
    ax.plot(n, np.exp(y - evidence), label='$\\pi=%.1f$' % pi)
ax.set_xscale('log')
# ax.set_yscale('log')
# ax.legend(loc='lower left')
ax.set_xlim((1, n.max()))

ax.set_yticks(np.arange(4))
# ax.grid()
for k, spine in ax.spines.items():
    if k in ['top', 'right']:
        spine.set_visible(False)
    if k == 'bottom':
        spine.set_bounds(1, 10**np.floor(np.log10(n.max())))

ax = axes[1]
mrg = .2
for pi in pivals:
    y = stats.beta(pi * nu, (1 - pi) * nu).pdf(p)
    ax.plot([-mrg, pi], [y, y])

x = np.linspace(0, 1, 200)
y = stats.beta(x * nu, (1 - x) * nu).pdf(p)
ax.plot(x, y, color='k', linewidth=2)
ax.fill_between(x, y, color='k', alpha=.1)

# ax.set_yscale('log')
# ax.set_yl

ax.set_xbound([0, 1])
ax.set_xlim((-mrg, 1.02))
ax.set_xticks(np.linspace(0, 1, 6))
ax.set_yticks([])

ax.set_xlabel('prior mean $\pi$')

# ax.grid()
for k, spine in ax.spines.items():
    if k == 'bottom':
        spine.set_bounds(0, 1)
    else:
        spine.set_visible(False)

for ax in axes:
    ax.set_ylim(0, 3)
pass

fig.tight_layout()
fig.savefig(figdir / 'likelihood-large-n-limit.jpg', dpi=300)